# WHAT TYPE OF PASSENGERS WERE MORE LIKELY TO SURVIVE THE TITANIC DISASTER?

# 1.Question

什么样的人在泰坦尼克号中更容易存活？

# 2.Understanding Data

## 2.1 Collecting Data

## 2.2 Loading Packages and Data

In [4]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

In [5]:
train = pd.read_csv("./train.csv")
test  = pd.read_csv("./test.csv")
print ('训练数据集:',train.shape,'测试数据集:',test.shape)

FileNotFoundError: [Errno 2] No such file or directory: './train.csv'

In [ ]:
rowNum_train=train.shape[0]
rowNum_test=test.shape[0]
print('kaggle训练数据集有多少行数据：',rowNum_train,
     ',kaggle测试数据集有多少行数据：',rowNum_test,)

In [ ]:
full = train.append( test , ignore_index = True )

print ('合并后的数据集:',full.shape)

## 2.3 Data Exploration

In [ ]:
full.head()

In [ ]:
full.describe()

In [ ]:
full.info()

# 3.Data Preparation 

## 3.1 Light Data Preprocessing

In [ ]:
print('处理前：')
full.info()

full['Age']=full['Age'].fillna( full['Age'].mean() )

full['Fare'] = full['Fare'].fillna( full['Fare'].mean() )
print('处理红后：')
full.info()

In [ ]:
full.head()

In [ ]:
full['Embarked'].head()

In [ ]:
full['Embarked'].value_counts()

In [ ]:
full['Embarked'] = full['Embarked'].fillna( 'S' )

In [ ]:
full['Cabin'].head()

In [ ]:
full['Cabin'] = full['Cabin'].fillna( 'U' )

In [ ]:
full.head()

In [6]:
full.info()

NameError: name 'full' is not defined

## 3.2 Data Classification

In [ ]:
full.info()

In [ ]:
full['Sex'].head()

In [ ]:
sex_mapDict={'male':1,
            'female':0}

full['Sex']=full['Sex'].map(sex_mapDict)
full.head()

#### Embarked

In [ ]:
full['Embarked'].head()

In [ ]:
embarkedDf = pd.DataFrame()

embarkedDf = pd.get_dummies( full['Embarked'] , prefix='Embarked' )
embarkedDf.head()

In [ ]:
full = pd.concat([full,embarkedDf],axis=1)

full.drop('Embarked',axis=1,inplace=True)
full.head()


#### Pclass

In [ ]:
pclassDf = pd.DataFrame()

pclassDf = pd.get_dummies( full['Pclass'] , prefix='Pclass' )
pclassDf.head()

In [ ]:
full = pd.concat([full,pclassDf],axis=1)

full.drop('Pclass',axis=1,inplace=True)
full.head()

#### Name

In [ ]:
full[ 'Name' ].head()

In [ ]:
name1='Braund, Mr. Owen Harris'

#Mr. Owen Harris
str1=name1.split( ',' )[1] 

#Mr.
str2=str1.split( '.' )[0]

str3=str2.strip()

In [ ]:
def getTitle(name):
    str1=name.split( ',' )[1] #Mr. Owen Harris
    str2=str1.split( '.' )[0]#Mr
    
    str3=str2.strip()
    return str3

In [ ]:
titleDf = pd.DataFrame()
titleDf['Title'] = full['Name'].map(getTitle)
titleDf.head()

In [ ]:
title_mapDict = {
                    "Capt":       "Officer",
                    "Col":        "Officer",
                    "Major":      "Officer",
                    "Jonkheer":   "Royalty",
                    "Don":        "Royalty",
                    "Sir" :       "Royalty",
                    "Dr":         "Officer",
                    "Rev":        "Officer",
                    "the Countess":"Royalty",
                    "Dona":       "Royalty",
                    "Mme":        "Mrs",
                    "Mlle":       "Miss",
                    "Ms":         "Mrs",
                    "Mr" :        "Mr",
                    "Mrs" :       "Mrs",
                    "Miss" :      "Miss",
                    "Master" :    "Master",
                    "Lady" :      "Royalty"
                    }

titleDf['Title'] = titleDf['Title'].map(title_mapDict)

titleDf = pd.get_dummies(titleDf['Title'])
titleDf.head()

In [ ]:
full = pd.concat([full,titleDf],axis=1)

full.drop('Name',axis=1,inplace=True)
full.head()

### Cabin

In [ ]:
sum = lambda a,b: a + b
 
print ("相加后的值为 : ", sum(10,20))

In [ ]:
full['Cabin'].head()

In [ ]:
cabinDf = pd.DataFrame()
full[ 'Cabin' ] = full[ 'Cabin' ].map( lambda c : c[0] )

cabinDf = pd.get_dummies( full['Cabin'] , prefix = 'Cabin' )

cabinDf.head()

In [ ]:
full = pd.concat([full,cabinDf],axis=1)

full.drop('Cabin',axis=1,inplace=True)
full.head()

### Family Size

In [ ]:
familyDf = pd.DataFrame()

familyDf[ 'FamilySize' ] = full[ 'Parch' ] + full[ 'SibSp' ] + 1

#if 条件为真的时候返回if前面内容，否则返回0
familyDf[ 'Family_Small' ] = familyDf[ 'FamilySize' ].map( lambda s : 1 if s == 1 else 0 )
familyDf[ 'Family_Medium' ]  = familyDf[ 'FamilySize' ].map( lambda s : 1 if 2 <= s <= 4 else 0 )
familyDf[ 'Family_Large' ]  = familyDf[ 'FamilySize' ].map( lambda s : 1 if 5 <= s else 0 )

familyDf.head()

In [ ]:
full = pd.concat([full,familyDf],axis=1)
full.head()

In [ ]:
full.shape

## 3.3 Feature Engineering

In [ ]:
corrDf = full.corr() 
corrDf

In [ ]:
corrDf['Survived'].sort_values(ascending =False)

In [ ]:
full_X = pd.concat( [titleDf,
                     pclassDf,
                     familyDf,
                     full['Fare'],
                     cabinDf,
                     embarkedDf,
                     full['Sex']
                    ] , axis=1 )
full_X.head()

# 4.Basic Model Building

## 4.1 Modeling

In [ ]:
sourceRow=891

source_X = full_X.loc[0:sourceRow-1,:]
source_y = full.loc[0:sourceRow-1,'Survived']   
pred_X = full_X.loc[sourceRow:,:]

In [ ]:
print('原始数据集有多少行:',source_X.shape[0])
print('原始数据集有多少行:',pred_X.shape[0])

In [ ]:
from sklearn.cross_validation import train_test_split 

train_X, test_X, train_y, test_y = train_test_split(source_X ,
                                                    source_y,
                                                    train_size=.8)
print ('原始数据集特征：',source_X.shape, 
       '训练数据集特征：',train_X.shape ,
      '测试数据集特征：',test_X.shape)

print ('原始数据集标签：',source_y.shape, 
       '训练数据集标签：',train_y.shape ,
      '测试数据集标签：',test_y.shape)

In [ ]:
source_y.head()

## 4.2 Choosing Algorithm

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()

## 4.3 Traning Model

In [ ]:
model.fit( train_X , train_y )

## 5.Evaluation

In [ ]:
model.score(test_X , test_y )

# 6.Deployment

In [ ]:
pred_Y = model.predict(pred_X)

pred_Y=pred_Y.astype(int)

passenger_id = full.loc[sourceRow:,'PassengerId']

predDf = pd.DataFrame( 
    { 'PassengerId': passenger_id , 
     'Survived': pred_Y } )
predDf.shape
predDf.head()

predDf.to_csv( 'titanic_pred.csv' , index = False )